# 线性回归的简洁实现

在过去的几年里，出于对深度学习强烈的兴趣，
许多公司、学者和业余爱好者开发了各种成熟的开源框架。
这些框架可以自动化基于梯度的学习算法中重复性的工作。
在`sec_linear_scratch`中，我们只运用了：
（1）通过张量来进行数据存储和线性代数；
（2）通过自动微分来计算梯度。
实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用，
现代深度学习库也为我们实现了这些组件。

本节将介绍如何**通过使用深度学习框架来简洁地实现**
`sec_linear_scratch`中的**线性回归模型**。

## 生成数据集

与`sec_linear_scratch`中类似，我们首先**生成数据集**。

In [1]:
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

我们可以**调用框架中现有的API来读取数据**。
我们将`features`和`labels`作为API的参数传递，并通过数据迭代器指定`batch_size`。
此外，布尔值`is_train`表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)  # * 操作符用于解包（unpack）列表或元组的元素；将其中的每一个元素作为独立的参数传给该函数
    return data.DataLoader(dataset, batch_size, shuffle=is_train)  # shuffle 表示是否需要打乱数据

*data_arrays 通过解包，将 data_arrays 中的每个元素分别传递给 data.TensorDataset。

这使得 data.TensorDataset 函数可以接收多个输入，并将它们组合成一个 dataset 对象，用于后续的数据加载。

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

使用`data_iter`的方式与我们在`sec_linear_scratch`中使用`data_iter`函数的方式相同。为了验证是否正常工作，让我们读取并打印第一个小批量样本。
与`sec_linear_scratch`不同，这里我们使用`iter`构造Python迭代器，并使用`next`从迭代器中获取第一项。

In [11]:
next(iter(data_iter))  # 每次获取其中一个 batch 的数据

[tensor([[-1.3003,  0.3906],
         [ 1.2297, -0.9080],
         [ 1.6924, -1.0978],
         [ 0.6375, -0.1914],
         [-1.4082,  0.6113],
         [ 1.2594, -0.6838],
         [-0.6714,  0.2089],
         [ 0.8279, -1.2986],
         [-0.2496,  1.6085],
         [-0.6968, -0.1969]]),
 tensor([[ 0.2722],
         [ 9.7724],
         [11.2857],
         [ 6.1427],
         [-0.6962],
         [ 9.0473],
         [ 2.1231],
         [10.2955],
         [-1.7734],
         [ 3.4717]])]

`data_iter`的数据类型是 torch.utils.data.DataLoader，同样可以**像迭代器一样遍历**`DataLoader`对象，**每次返回一个批次的数据**。

In [13]:
for X, y in iter(data_iter):
    print(X, '\n', y)
    break

tensor([[-0.1391, -0.1412],
        [ 0.4221, -0.1557],
        [-0.9546, -1.0746],
        [ 0.1864, -0.2919],
        [ 0.8717,  0.3961],
        [-1.4560,  2.4827],
        [-0.2395, -0.7249],
        [-0.9760,  0.3954],
        [ 2.0230, -1.8391],
        [ 0.8090,  1.4358]]) 
 tensor([[ 4.3919],
        [ 5.5651],
        [ 5.9486],
        [ 5.5703],
        [ 4.5843],
        [-7.1483],
        [ 6.2047],
        [ 0.9032],
        [14.4948],
        [ 0.9240]])


## 定义模型

当我们在`sec_linear_scratch`中实现线性回归时，
我们明确定义了模型参数变量，并编写了计算的代码，这样通过基本的线性代数运算得到输出。
但是，如果模型变得更加复杂，且当我们几乎每天都需要实现模型时，自然会想简化这个过程。
这种情况类似于为自己的博客从零开始编写网页。
做一两次是有益的，但如果每个新博客就需要工程师花一个月的时间重新开始编写网页，那并不高效。

对于标准深度学习模型，我们可以**使用框架的预定义好的层**。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。
我们首先定义一个模型变量`net`，它是一个`Sequential`类的实例。
`Sequential`类将多个层串联在一起。
当给定输入数据时，`Sequential`实例将数据传入到第一层，
然后将第一层的输出作为第二层的输入，以此类推。
在下面的例子中，我们的模型只包含一个层，因此实际上不需要`Sequential`。
但是由于以后几乎所有的模型都是多层的，在这里使用`Sequential`会让你熟悉“标准的流水线”。

回顾`fig_single_neuron`中的单层网络架构，
这一单层被称为*全连接层*（fully-connected layer），
因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。

对于线性回归，**每个输入都与每个输出（在本例中只有一个输出）相连**， 我们将这种变换（`fig_single_neuron`图中的输出层）称为全连接层（fully-connected layer）或称为稠密层（dense layer）。

在PyTorch中，全连接层在`Linear`类中定义。
值得注意的是，我们将两个参数传递到`nn.Linear`中。
第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。


In [14]:
# nn是神经网络 neural network 的缩写
from torch import nn

# sequential 可以理解为 list of layers（许多层按顺序排在一起）
net = nn.Sequential(nn.Linear(2, 1))  # 输入的维度是 2（两个特征变量），输出的维度是 1（一个输出变量）；X * W，矩阵乘法的顺序很重要
# 这里的维度即为 W 的维度参数

## **初始化模型参数**

在使用`net`之前，我们需要初始化模型参数。
如在线性回归模型中的权重和偏置。
深度学习框架通常有预定义的方法来初始化参数。
在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，
偏置参数将初始化为零。

正如我们在构造`nn.Linear`时指定输入和输出尺寸一样，
现在我们能直接访问参数以设定它们的初始值。
我们通过`net[0]`选择网络中的第一个图层，
然后使用`weight.data`和`bias.data`方法访问参数。
我们还可以使用替换方法`normal_`和`fill_`来重写参数值。


In [7]:
# 初始化模型，随机设置初始参数（w 和 b）
# 因为只有一层，可以通过 net[0] 来访问上述定义的 layer
net[0].weight.data.normal_(0, 0.01)  # w 权重，normal_ 使用正态分布随机生成
net[0].bias.data.fill_(0)  # b 偏置，fill_ 使用 0 替换

tensor([0.])

## 定义损失函数


计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数。
默认情况下，它返回所有样本损失的平均值。

In [15]:
loss = nn.MSELoss()

## 定义优化算法


小批量随机梯度下降算法是一种优化神经网络的标准工具，
PyTorch在`optim`模块中实现了该算法的许多变种。
当我们**实例化一个`SGD`实例**时，我们要指定优化的参数
可通过`net.parameters()`从我们的模型中获得以及优化算法所需的超参数字典。
小批量随机梯度下降只需要设置`lr`值，这里设置为0.03。

In [16]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)  # optim - optimize
# net.parameters() 得到网络的所有参数

## 训练

通过深度学习框架的高级API来实现我们的模型只需要相对较少的代码。
我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。
当我们需要更复杂的模型时，高级API的优势将大大增加。
当我们有了所有的基本组件，[**训练过程代码与我们从零开始实现时所做的非常相似**]。

回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（`train_data`），
不停地从中获取一个小批量的输入和相应的标签。
对于每一个小批量，我们会进行以下步骤:

* 通过调用`net(X)`生成预测并计算损失`l`（前向传播）。
* 通过进行反向传播来计算梯度。
* 通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。

In [10]:
num_epochs = 3  # 训练过程循环3次
for epoch in range(num_epochs):  # 在每个 epoch 中，模型会遍历一次所有的训练数据

    for X, y in data_iter:
        l = loss(net(X), y)  # 前向传播
        trainer.zero_grad()  # trainer 是前面定义的 SGD 优化器；对梯度清零（因为 pytorch 的梯度是累加的）
        l.backward()  # 反向传播，计算损失 l 对参数的梯度
        trainer.step()  # 用计算出的梯度更新模型参数（梯度下降法更新参数）
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000248
epoch 2, loss 0.000103
epoch 3, loss 0.000103


下面我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。
要访问参数，我们首先从`net`访问所需的层，然后读取该层的权重和偏置。
正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [17]:
w = net[0].weight.data
print('w的估计误差: ', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差: ', true_b - b)

w的估计误差： tensor([ 2.4162, -3.4136])
b的估计误差： tensor([3.5185])


## 批量大小选择

batch_size（批量大小）指的是每次更新模型参数时，使用的样本数量。

**往往 batch_size 越小，收敛质量效果越好。**

这是因为随机梯度下降理论上来说是带来了噪音（每次选取小批量的样本，而非是全部样本）；**每次采样的样本越小，噪音越多**。但是**噪音对神经网络是有利的**，

**更新频率更高**
- 当 batch_size 较小（例如，甚至是 1，也称为随机梯度下降，SGD），模型参数会更频繁地更新。
- 频繁的参数更新意味着模型能更快地适应数据的变化，有助于在参数空间中找到更好的路径，尽早进入收敛状态。

**更具噪声的梯度估计**
- 较小的 `batch_size` 使得梯度估计中包含更多噪声。这些噪声可以避免模型陷入局部最优，帮助模型跳出局部最优解。
- 这种随机性有助于在复杂的损失面上找到全局最优解，尤其是在非凸优化问题中（如神经网络训练）。

**更好的正则化效果**
- 小批量训练天然具有正则化效果，可以防止过拟合。由于每次梯度计算只使用少量样本，这使得模型在训练过程中无法完全记住所有样本，从而更注重学习数据的普遍模式。
- 这种正则化效果类似于加入了随机噪声，有助于提高模型的泛化能力。

## 小结

* 我们可以使用PyTorch的高级API更简洁地实现模型。
* 在PyTorch中，`data`模块提供了数据处理工具，`nn`模块定义了大量的神经网络层和常见损失函数。
* 我们可以通过`_`结尾的方法将参数替换，从而初始化参数。


## 练习

1. 如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？
2. 查看深度学习框架文档，它们提供了哪些损失函数和初始化方法？用Huber损失代替原损失，即
    $$l(y,y') = \begin{cases}|y-y'| -\frac{\sigma}{2} & \text{ if } |y-y'| > \sigma \\ \frac{1}{2 \sigma} (y-y')^2 & \text{ 其它情况}\end{cases}$$
3. 如何访问线性回归的梯度？

[Discussions](https://discuss.d2l.ai/t/1781)
